<a href="https://colab.research.google.com/github/Ghiles1010/Recommendation-system/blob/master/Semantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install owlready2
from owlready2 import *
import json
import os
import re

from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/PFE STUFF/Yelp_DB")

     |████████████████████████████████| 20.8MB 1.4MB/s 
  Created wheel for owlready2: filename=Owlready2-0.24-cp36-cp36m-linux_x86_64.whl size=20263016 sha256=9577de60049cfc807a48fbbdff27e2151128722614e15f0debcffe3f41202d05
  Stored in directory: /root/.cache/pip/wheels/66/87/0b/7a97bd19c9dc817b279aadb5dd920ce3a77050e2ab5a413f43
Successfully built owlready2
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
file_u = open("Users_cat_Yelp.json")
l_u = file_u.readlines()
file_u.close()

f_b = open("Business_Yelp.json")
l_b = f_b.readlines()
f_b.close()

users = {}
for i in l_u:
  js = json.loads(i)
  users[js["user_id"]] = js

prests = {}
for i in l_b:
  js = json.loads(i)
  prests[js["business_id"]] = js

In [ ]:
dist = {} #we save distances to not recalculate them
def wp_sim(c1, c2):

	if ( dist.get( frozenset( (c1, c2) ) ) != None ):

		return dist[frozenset( (c1, c2) ) ]

	else:
		anc1=c1.ancestors()
		anc2=c2.ancestors()
		
		cpt=0
		
		if(c1!=c2):
			for i in anc1:
				if i in anc2:
					cpt=cpt+1

		else:
			return 1

		N=cpt-1	
		N1=len(anc1)-1
		N2=len(anc2)-1
		

		r=((2*N)/(N1+N2))
		dist[frozenset((c1, c2))] = r
		return r

def onto_class(c, onto): #returns the owlready2 class
	if(isinstance(c, owlready2.entity.ThingClass)):
		return c
	else:
		#print(str(c)[50:-1])
		#s = re.match("(.+?\()(.+?\.)(.+[A-Za-z_])",str(c))
		s = str(c)[50:-1]
		return onto[s]
	

def interest(u, v):

	interest=0
	cpt=0

	#print(u.is_a)	
	#print(v.is_a)
	
	for cat in v.is_a:
		if isinstance(onto_class(cat, onto), onto["Category"]):

			selected = ""
			max = 0

			for cat2 in u.is_a:
				if isinstance(onto_class(cat2, onto), onto["Category"]):
					
					sim = wp_sim(onto_class(cat, onto), onto_class(cat2, onto))

					if sim > max:

						max = sim
						selected = str(cat2)[50:-1]
			
			if max == 0:
				print("error", u.is_a, u.name, v.is_a, v.name)
			else:
				
				weight = users[u.name]["categories"][selected]
				interest += max * weight
				cpt += weight


	#print(interest)
	#print(cpt)
	
	return interest/cpt


In [ ]:
onto = get_ontology("Ontology_filled.owl").load()

In [ ]:
print(onto["Sandwiches"])
print(onto["Themed_Cafes"])

Ontology_filled.Sandwiches
Ontology_filled.Themed_Cafes


In [ ]:
import json
from math import *

file_r  = open("Review_Yelp.json")
l_r = file_r.readlines()
file_r.close()

print("loaded")

cpt = 0
results = []
inter = []
y_true = []
y_pred = []

for i in range(100):

  rv = json.loads(l_r[i])

  id_u = rv["user_id"]
  id_b = rv["business_id"]

  client = onto.Client(id_u)
  prest = onto.Service(id_b)

  interested = interest(client, prest)

  if interested > 1:
    print("error")
  prediction = interested * prests[id_b]["stars"]

  y_true.append(rv["stars"])
  y_pred.append( prediction )

  inter.append( (id_u, id_b, interested) )
  results.append(  (id_u, id_b, prediction) )

  cpt+=1

  if (cpt%1000)==0:
    print("cpt = ", cpt)  


loaded


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print("mae = ", mean_absolute_error(y_true, y_pred))
print("rmse = ", sqrt(mean_squared_error(y_true, y_pred)))

mae =  0.82
rmse =  1.0723805294763609


In [ ]:
o = open("results_sem.txt",'w')

for i in results:
  o.write(i[0]+" "+i[1]+" "+str(i[2])+"\n")
o.close()

In [ ]:
o = open("interest.txt",'w')

for i in inter:
  o.write(i[0]+" "+i[1]+" "+str(i[2])+"\n")
o.close()

In [ ]:
o = open("interest.txt")

l_o = o.readlines()

for i in l_o:
  
  t = float(i.split()[2])


  if t != 1.0:
    print(t)
o.close()

#Semantic with user-user


In [ ]:
import numpy as np
import pandas as pd

file_r  = open("Review_Yelp.json")
l_r = file_r.readlines()
file_r.close()

ratings = []

n_users = {}
n_bus = {}

np.zeros

for i in l_r:

  rv = json.loads(i)

  idu = rv["user_id"]
  idb = rv["business_id"]
  stars = rv["stars"]

  n_users[idu] = n_users.get(idu, 0) + 1
  n_bus[idb] = n_bus.get(idb, 0) + 1


  ratings.append(  [idu, idb, stars] )



In [ ]:
height = len(n_users)
width = len(n_bus)
matrix = np.zeros(shape=(height, width))
matrix = pd.DataFrame(matrix, columns = list(n_bus.keys()), index = list(n_users.keys()))

for i in ratings:

  matrix.loc[i[0], i[1]] = i[2]

In [ ]:
from sklearn.metrics import jaccard_score

def sim_user_user(user1, user2):

  user1 = np.array(matrix.loc[user1,:])
  user2 = np.array(matrix.loc[user2,:])

  user1 = [ 1 if i>0 else 0 for i in user1]
  user2 = [ 1 if i>0 else 0 for i in user2]

  return jaccard_score(user1, user2)

In [ ]:
dic_user_user = {}

i = 0
cpt = i
print(height)
while i < height:
  j = i
  while j < height:
    user1 = matrix.index[i]
    user2 = matrix.index[j]
    dist = sim_user_user(user1, user2)
    s = frozenset((user1, user2))
    dic_user_user[s] = dist
  
  j += 1
  cpt += height
  print(cpt)
i += 1

In [ ]:
print(i)

NameError: ignored

In [ ]:
def note(user1, users, business):

  weights = 0
  somme = 0
  for user in users.index:

    if user != user1:
      w = sim_user_user(user1, user)
      somme += w * users.loc[user,business]
      weights += w
  
  if weights != 0:
    return somme/weights
  
  else:
    return matrix.loc[user1,business]


In [ ]:
import json
from math import *

file_r  = open("Review_Yelp.json")
l_r = file_r.readlines()
file_r.close()

print("loaded")

cpt = 0
results = []
inter = []
y_true = []
y_pred = []

for i in range(100000):

  rv = json.loads(l_r[i])

  id_u = rv["user_id"]
  id_b = rv["business_id"]

  client = onto.Client(id_u)
  prest = onto.Service(id_b)

  interested = interest(client, prest)

  if interested > 1:
    print("error")

  sim_users = matrix[matrix[id_b] != 0] 

  n = note(id_u, sim_users, id_b)

  prediction = interested * n

  y_true.append(rv["stars"])
  y_pred.append( prediction )

  inter.append( (id_u, id_b, interested) )
  results.append(  (id_u, id_b, prediction) )

  cpt+=1

  if (cpt%10)==0:
    print("cpt = ", cpt)  

  if (cpt%10)==0:
    print("mae = ", mean_absolute_error(y_true, y_pred))
    print("rmse = ", sqrt(mean_squared_error(y_true, y_pred)))

loaded
cpt =  10
mae =  0.5964639843095844
rmse =  0.8773605474492234
cpt =  20
mae =  0.6463289869935702
rmse =  0.8717256128224383
cpt =  30
mae =  0.8894255623639119
rmse =  1.1158401836130372
cpt =  40
mae =  0.8012018087274562
rmse =  1.0192570402885124
cpt =  50
mae =  0.7862377177828123
rmse =  0.9917582484413897
cpt =  60
mae =  0.8058616000959756
rmse =  1.0143813130778339
cpt =  70
mae =  0.8315249804503592
rmse =  1.0443587387049653
cpt =  80
mae =  0.8526650549863206
rmse =  1.070507360677711
cpt =  90
mae =  0.8698569616090676
rmse =  1.0960082538499747
cpt =  100
mae =  0.865620514047732
rmse =  1.0957027856440573
cpt =  110
mae =  0.8468415003994496
rmse =  1.0690409003296626
cpt =  120
mae =  0.8438459575053876
rmse =  1.0560487589783227
cpt =  130
mae =  0.8767049754105859
rmse =  1.1015279676359264
cpt =  140
mae =  0.8841126042934689
rmse =  1.1047822568565204
cpt =  150
mae =  0.9024933584922481
rmse =  1.1201259232280916
cpt =  160
mae =  0.8891989955689038
rmse = 

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
print("mae = ", mean_absolute_error(y_true, y_pred))
print("rmse = ", sqrt(mean_squared_error(y_true, y_pred)))

mae =  0.4573414272356595
rmse =  0.5756154557991622


In [ ]:
print(y_true[:10])
print(y_pred[:10])

[4.0, 4.0, 4.0, 4.0, 1.0, 2.0, 4.0, 5.0, 4.0, 1.0]
[3.8882927548272663, 3.9962808208198775, 3.9457623063907645, 4.062215907418183, 1.636879974342237, 2.5749167416301066, 4.166333580784723, 4.363046128382418, 4.010414583552971, 1.6284259750214287]


In [ ]:
print(cpt)

57
